<a href="https://colab.research.google.com/github/Favouriteg2/GenAI-class-practical/blob/main/Copy_of_GEN_AI_FOR_SPEECH_HANDS_ON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **YARNGPT.ai TTS API PLAYGROUND**

In [1]:
import os
from google.colab import userdata

yarn_api = userdata.get('yarngpt-api-token')

In [2]:
import requests #this is used to make request to the api url

API_URL = "https://yarngpt.ai/api/v1/tts"
API_KEY = yarn_api

headers = {
    "Authorization": f"Bearer {API_KEY}"
}

payload = {
    "text": "Ólajide sọ pé ó yẹ kí ìjọba tètè dá sí àtúnṣe àwọn òòfísì náà kó tó pẹ́ ju kí ìjàmbá má baà wáyé sí àwọn tó ń ṣiṣẹ́ nínú rẹ̀. Aṣòfin náà, tó jẹ́ alága ìgbìmọ̀ tó ń rí sí ọ̀rọ̀ ìjọba ìbílẹ̀ àti ọ̀rọ̀ oyè jíjẹ sọ pé púpọ̀ nínú àwọn òòfísì tó wà láwọn ìjọba ìbílẹ̀ káàkiri ìpínlẹ̀ Ọ̀yọ ni kò ṣe é ló mọ́.",
    "voice": "Idera",
}

response = requests.post(API_URL, headers=headers, json=payload, stream=True)

if response.status_code == 200:
    with open("output.mp3", "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Audio file saved as output.mp3")
else:
    print(f"Error: {response.status_code}")
    print(response.json())

Audio file saved as output.mp3


In [3]:
from IPython.display import Audio

# After saving the file
Audio("output.mp3")

In [4]:
import requests

API_URL = "https://yarngpt.ai/api/v1/tts"
API_KEY = yarn_api

headers = {
    "Authorization": f"Bearer {API_KEY}"
}

payload = {
    "text": "Generative AI class for everyone, we should all join",
    "voice": "Wura",
}

response = requests.post(API_URL, headers=headers, json=payload, stream=True)

if response.status_code == 200:
    with open("output_wura.wav", "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Audio file saved as output_wura.wav")
else:
    print(f"Error: {response.status_code}")
    print(response.json())

Audio file saved as output_wura.wav


In [ ]:
Audio("output_wura.wav")

### **SPITCH.APP TTS DEMO**

In [5]:
!pip install spitch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.7 MB/s eta 0:00:00


In [9]:
spitch_api = userdata.get("spitch-token")

In [10]:
from spitch import Spitch

client = Spitch(
    api_key=spitch_api,
)

response = client.speech.generate(
    language="en",
    text="Generative AI class for everyone, we should all join",
    voice="lucy",
)

# Get the audio bytes
audio_bytes = response.read()

with open("output.wav", "wb") as f:
    f.write(audio_bytes)

print("Voice generated and saved to output.wav")


Voice generated and saved to output.wav


In [11]:
Audio("output.wav")

In [12]:
# Transcribe audio file to text
response = client.speech.transcribe(
    content=open("output.wav", "rb"),
    language="en"  # Can be "en", "yo", "ig", "ha"
)

# Print the transcribed text
print(response.text)

Generative AI class for everyone, we should all join.


In [ ]:
# Transcribe audio file to text
response = client.speech.transcribe(
    content=open("AUDIO-2025-11-26-19-45-21.m4a", "rb"),
    language="en"  # Can be "en", "yo", "ig", "ha"
)

# Print the transcribed text
print(response.text)

I'm able to send it. I'm recording this checking if I can send it to my laptop. Let's see.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive

### **Mini project: podcast with yarngpt**

In [13]:
!pip install pydub

In [14]:
import io
import requests
import gradio as gr
from pydub import AudioSegment

API_URL = "https://yarngpt.ai/api/v1/tts"
VOICES = ["Idera", "Femi", "Emma", "Zainab"]

def call_tts(text, voice, api_key):
    """Call YarnGPT TTS API with user's key"""
    if not api_key or not api_key.strip():
        return None, "Please enter your API key!"

    headers = {"Authorization": f"Bearer {api_key.strip()}"}
    # Don't specify format, let API decide
    payload = {"text": text, "voice": voice}

    try:
        r = requests.post(API_URL, headers=headers, json=payload, stream=True, timeout=30)

        if r.status_code != 200:
            return None, f"API Error {r.status_code}: {r.text}"

        # Check content type
        content_type = r.headers.get('Content-Type', 'unknown')
        print(f"Content-Type: {content_type}")

        audio = io.BytesIO()
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                audio.write(chunk)

        audio.seek(0)  # Reset to beginning

        return audio.getvalue(), f"Success! Format: {content_type}"

    except Exception as e:
        return None, f"Error: {str(e)}"

def generate_conversation(api_key, conversation, voice1, voice2):
    """Generate audio for alternating conversation"""
    if not api_key or not api_key.strip():
        return None, "Please enter your YarnGPT API key first!"

    lines = [line.strip() for line in conversation.splitlines() if line.strip()]

    if not lines:
        return None, "Please enter some conversation text!"

    audio_segments = []
    status_msg = ""

    for i, line in enumerate(lines):
        voice = voice1 if i % 2 == 0 else voice2
        status_msg += f"🔄 Generating line {i+1}/{len(lines)} with {voice}...\n"

        audio_bytes, msg = call_tts(line, voice, api_key)

        if audio_bytes is None:
            return None, f"{status_msg}\nFailed at line {i+1}: {msg}"

        try:
            # Try loading without specifying format (let pydub detect)
            audio_io = io.BytesIO(audio_bytes)
            audio_segment = AudioSegment.from_file(audio_io)
            audio_segments.append(audio_segment)

            status_msg += f"Line {i+1}: {voice} - \"{line}\"\n"

        except Exception as e:
            return None, f"{status_msg}\nFailed to load audio at line {i+1}: {str(e)}"

    # Concatenate all audio segments
    if audio_segments:
        combined_audio = audio_segments[0]
        for segment in audio_segments[1:]:
            # Add a small pause between lines
            silence = AudioSegment.silent(duration=500)
            combined_audio = combined_audio + silence + segment

        # Export to bytes
        output = io.BytesIO()
        combined_audio.export(output, format="mp3", bitrate="128k")
        output.seek(0)

        full_status = status_msg + f"\n🎉 Generated and combined {len(audio_segments)} audio clips!"
        return output.getvalue(), full_status

    return None, "No audio generated"

# [Rest of the Gradio interface code stays the same...]
with gr.Blocks(title="YarnGPT Two Voice Chat", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎧 Two Voice Chat with YarnGPT")
    gr.Markdown("Create podcast-style conversations with alternating Nigerian voices!")

    with gr.Row():
        api_key_input = gr.Textbox(
            label="🔑 Your YarnGPT API Key",
            placeholder="Enter your API key from yarngpt.ai",
            type="password",
            info="Get your free API key at https://yarngpt.ai"
        )

    gr.Markdown("---")
    gr.Markdown("### Voice Selection")

    with gr.Row():
        voice1 = gr.Dropdown(VOICES, label="🎤 Voice 1", value="Idera")
        voice2 = gr.Dropdown(VOICES, label="🎤 Voice 2", value="Femi")

    conversation = gr.Textbox(
        label="Conversation Script",
        lines=8,
        value="Hello, welcome to our show.\nThank you, I am happy to be here."
    )

    generate_btn = gr.Button("🎙️ Generate Audio", variant="primary", size="lg")

    gr.Markdown("---")

    with gr.Row():
        audio_output = gr.Audio(label="🔊 Full Conversation Audio")
        status_output = gr.Textbox(label="📊 Status", lines=10)

    generate_btn.click(
        fn=generate_conversation,
        inputs=[api_key_input, conversation, voice1, voice2],
        outputs=[audio_output, status_output]
    )


demo.launch(debug=True)

/tmp/ipython-input-424665332.py:92: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="YarnGPT Two Voice Chat", theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://30f923783b36ac0257.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Content-Type: audio/wav
Content-Type: audio/wav
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://30f923783b36ac0257.gradio.live


### **Bringing it Together**

In [ ]:
import io
import requests
import gradio as gr
from pydub import AudioSegment
from huggingface_hub import InferenceClient

# APIs
YARNGPT_API_URL = "https://yarngpt.ai/api/v1/tts"
VOICES = ["Idera", "Femi", "Emma", "Zainab"]

# ==================== CONVERSATION GENERATION ====================

def generate_conversation_with_llm(hf_api_key, topic, num_turns=4):
    """Use Hugging Face InferenceClient to generate a conversation"""

    if not hf_api_key or not hf_api_key.strip():
        return None, "Please enter your Hugging Face API key!"

    if not topic or not topic.strip():
        return None, "Please describe what the conversation should be about!"

    try:
        # Initialize the InferenceClient
        client = InferenceClient(
            model="meta-llama/Llama-3.2-3B-Instruct",
            token=hf_api_key.strip()
        )

        # Create chat messages
        messages = [
            {
                "role": "system",
                "content": f"""You are a conversation writer. Generate natural dialogues between two people.
Format: Write ONLY the dialogue lines, one per line, with no speaker labels or names.
Each line should be conversational and concise (1-2 sentences).
Generate exactly {num_turns} lines total, alternating between two speakers."""
            },
            {
                "role": "user",
                "content": f"""Generate a {num_turns}-line conversation about: {topic}

Example format:
Hello, welcome to the show!
Thank you for having me today.
Let's discuss your recent work.
I'd be happy to share my experience.

Now generate the conversation:"""
            }
        ]

        # Call chat completion
        response = client.chat_completion(
            messages=messages,
            max_tokens=500,
            temperature=0.7,
            top_p=0.9
        )

        # Extract generated text
        generated_text = response.choices[0].message.content.strip()

        if not generated_text:
            return None, "No conversation generated. Try a different topic."

        # Clean up the output
        lines = [line.strip() for line in generated_text.split('\n') if line.strip()]

        # Remove any speaker labels if present
        cleaned_lines = []
        for line in lines:
            # Skip instruction-like lines
            if line.startswith(('Example', 'Format', 'Note', 'Remember', 'Here', 'Sure')):
                continue
            # Remove speaker labels (anything before a colon)
            if ':' in line and len(line.split(':', 1)[0].split()) <= 3:
                line = line.split(':', 1)[1].strip()
            # Skip empty lines after cleaning
            if line:
                cleaned_lines.append(line)

        # Take only the requested number of turns
        conversation = '\n'.join(cleaned_lines[:num_turns])

        if not conversation:
            return None, "Could not extract valid conversation. Try again."

        return conversation, f"Generated {len(cleaned_lines[:num_turns])} lines of conversation!"

    except Exception as e:
        error_msg = str(e)

        # More helpful error messages
        if "401" in error_msg or "authorization" in error_msg.lower():
            return None, "Invalid API key. Check your Hugging Face token."
        elif "503" in error_msg or "loading" in error_msg.lower():
            return None, "Model is loading. Please wait 20-30 seconds and try again."
        else:
            return None, f"Error: {error_msg}"

# ==================== AUDIO GENERATION (same as before) ====================

def call_tts(text, voice, api_key):
    """Call YarnGPT TTS API with user's key"""
    if not api_key or not api_key.strip():
        return None, "Please enter your YarnGPT API key!"

    headers = {"Authorization": f"Bearer {api_key.strip()}"}
    payload = {"text": text, "voice": voice}

    try:
        r = requests.post(YARNGPT_API_URL, headers=headers, json=payload, stream=True, timeout=30)

        if r.status_code != 200:
            return None, f"API Error {r.status_code}: {r.text}"

        content_type = r.headers.get('Content-Type', 'unknown')
        print(f"Content-Type: {content_type}")

        audio = io.BytesIO()
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                audio.write(chunk)

        audio.seek(0)
        return audio.getvalue(), f"Success! Format: {content_type}"

    except Exception as e:
        return None, f"Error: {str(e)}"

def generate_audio_from_conversation(yarngpt_api_key, conversation, voice1, voice2):
    """Generate audio for alternating conversation"""
    if not yarngpt_api_key or not yarngpt_api_key.strip():
        return None, "Please enter your YarnGPT API key first!"

    lines = [line.strip() for line in conversation.splitlines() if line.strip()]

    if not lines:
        return None, "Please enter some conversation text!"

    audio_segments = []
    status_msg = ""

    for i, line in enumerate(lines):
        voice = voice1 if i % 2 == 0 else voice2
        status_msg += f"🔄 Generating line {i+1}/{len(lines)} with {voice}...\n"

        audio_bytes, msg = call_tts(line, voice, yarngpt_api_key)

        if audio_bytes is None:
            return None, f"{status_msg}\nFailed at line {i+1}: {msg}"

        try:
            audio_io = io.BytesIO(audio_bytes)
            audio_segment = AudioSegment.from_file(audio_io)
            audio_segments.append(audio_segment)

            status_msg += f"Line {i+1}: {voice} - \"{line}\"\n"

        except Exception as e:
            return None, f"{status_msg}\nFailed to load audio at line {i+1}: {str(e)}"

    if audio_segments:
        combined_audio = audio_segments[0]
        for segment in audio_segments[1:]:
            silence = AudioSegment.silent(duration=500)
            combined_audio = combined_audio + silence + segment

        output = io.BytesIO()
        combined_audio.export(output, format="mp3", bitrate="128k")
        output.seek(0)

        full_status = status_msg + f"\nGenerated and combined {len(audio_segments)} audio clips!"
        return output.getvalue(), full_status

    return None, "No audio generated"

# ==================== GRADIO INTERFACE ====================

with gr.Blocks(title="AI Conversation Generator + TTS", theme=gr.themes.Soft()) as demo:

    gr.Markdown("# 🤖🎙️ AI Conversation Generator with Voice")
    gr.Markdown("**Step 1:** Describe your conversation → **Step 2:** Generate dialogue → **Step 3:** Convert to audio!")

    # ========== API KEYS SECTION ==========
    gr.Markdown("## 🔑 API Keys")

    with gr.Row():
        hf_api_key = gr.Textbox(
            label="Hugging Face API Key",
            placeholder="Get free key at huggingface.co/settings/tokens",
            type="password",
            scale=1
        )
        yarngpt_api_key = gr.Textbox(
            label="YarnGPT API Key",
            placeholder="Get free key at yarngpt.ai",
            type="password",
            scale=1
        )

    gr.Markdown("---")

    # ========== STEP 1: CONVERSATION GENERATION ==========
    gr.Markdown("## 📝 Step 1: Generate Conversation")

    with gr.Row():
        topic_input = gr.Textbox(
            label="What should the conversation be about?",
            placeholder="Example: A podcast about AI in African healthcare, or a debate about climate change...",
            lines=3,
            scale=3
        )
        num_turns = gr.Slider(
            minimum=2,
            maximum=12,
            value=6,
            step=2,
            label="Number of lines",
            scale=1
        )

    generate_conv_btn = gr.Button("🤖 Generate Conversation", variant="secondary", size="lg")

    conversation_output = gr.Textbox(
        label="Generated Conversation (editable)",
        placeholder="Your AI-generated conversation will appear here...",
        lines=10,
        info="You can edit this before converting to audio!"
    )

    conv_status = gr.Textbox(label="Status", lines=2)

    gr.Markdown("---")

    # ========== STEP 2: VOICE SELECTION ==========
    gr.Markdown("## 🎤 Step 2: Select Voices")

    with gr.Row():
        voice1 = gr.Dropdown(
            VOICES,
            label="Voice 1 (Odd lines: 1, 3, 5...)",
            value="Idera"
        )
        voice2 = gr.Dropdown(
            VOICES,
            label="Voice 2 (Even lines: 2, 4, 6...)",
            value="Femi"
        )

    gr.Markdown("---")

    # ========== STEP 3: AUDIO GENERATION ==========
    gr.Markdown("## 🎙️ Step 3: Generate Audio")

    generate_audio_btn = gr.Button("🔊 Convert to Audio", variant="primary", size="lg")

    with gr.Row():
        audio_output = gr.Audio(label="🎧 Final Audio")
        audio_status = gr.Textbox(label="Audio Status", lines=10)

    gr.Markdown("---")

    # ========== EXAMPLES ==========
    gr.Markdown("### 💡 Example Topics")
    gr.Examples(
        examples=[
            ["A podcast discussing the future of AI in African education"],
            ["Two friends debating whether Python or JavaScript is better"],
            ["A doctor explaining malaria prevention to a patient in simple terms"],
            ["A tech entrepreneur pitching their startup idea to an investor"],
            ["Two students discussing their favorite Nollywood movies"],
        ],
        inputs=[topic_input],
        label="Click to try these examples"
    )


    # Generate conversation
    generate_conv_btn.click(
        fn=generate_conversation_with_llm,
        inputs=[hf_api_key, topic_input, num_turns],
        outputs=[conversation_output, conv_status]
    )

    # Generate audio
    generate_audio_btn.click(
        fn=generate_audio_from_conversation,
        inputs=[yarngpt_api_key, conversation_output, voice1, voice2],
        outputs=[audio_output, audio_status]
    )

demo.launch(debug=True)

/tmp/ipython-input-1877764029.py:180: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="AI Conversation Generator + TTS", theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://709a587e5d1e5f2096.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
